In [5]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [9]:
data = pd.read_csv('./bandwidth.csv', header=0,  index_col=0)
final = pd.DataFrame(data)

final.tail(10)

,experiment,start,end,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,bandwidth
205,653,2023-11-21 21:34:06.4672540,2023-11-21 22:04:06.4816988,general,16,8,15,16,8,4,0.75,0.5,166.092000
206,654,2023-11-21 22:07:40.2346165,2023-11-21 22:37:40.2385545,general,5,4,7,3,2,2,0.25,1.0,161.342425
207,655,2023-11-21 22:41:14.2028913,2023-11-21 23:11:14.2104657,general,5,4,7,3,2,2,0.25,1.0,107.576214
208,656,2023-11-21 23:14:48.2294649,2023-11-21 23:44:48.2363093,general,5,4,7,3,2,2,0.25,1.0,218.079171
209,657,2023-11-21 23:48:22.6068989,2023-11-22 00:18:22.6099262,general,6,4,8,7,5,4,0.25,3.0,343.137756
210,658,2023-11-22 00:21:57.0236579,2023-11-22 00:51:57.0268426,general,6,4,8,7,5,4,0.25,3.0,41.519371
211,661,2023-11-22 02:02:40.2343885,2023-11-22 02:32:40.2368451,general,6,3,8,2,2,4,0.25,0.5,4.343883
212,662,2023-11-22 02:36:14.2823737,2023-11-22 03:06:14.2932442,general,6,3,8,2,2,4,0.25,0.5,37.513662
213,663,2023-11-22 03:09:48.5789046,2023-11-22 03:39:48.5860069,general,21,16,24,5,16,8,0.50,1.0,45.392599
214,664,2023-11-22 03:43:22.2716142,2023-11-22 04:13:22.2769946,general,21,16,24,5,16,8,0.50,1.0,6.162759


In [10]:
data = pd.read_csv('../datasets/timeSeries1s.csv', header=0,  index_col=0)
message = pd.DataFrame(data)

idx = [2.0]
message = message[message["type"].isin(idx)]
message = message.drop(columns=["type", "identifier"])

message.head(10)

,experiment,_time,count
1005,618,2023-11-16 21:11:58+00:00,1913
1006,618,2023-11-16 21:12:00+00:00,371
1007,618,2023-11-16 21:12:01+00:00,1888
1008,618,2023-11-16 21:12:03+00:00,365
1009,618,2023-11-16 21:12:04+00:00,1922
1010,618,2023-11-16 21:12:06+00:00,366
1011,618,2023-11-16 21:12:07+00:00,1898
1012,618,2023-11-16 21:12:09+00:00,372
1013,618,2023-11-16 21:12:10+00:00,1922
1014,618,2023-11-16 21:12:12+00:00,247


In [12]:
avgOve = message.groupby(['experiment']).agg({'count':['sum']})
avgOve.columns = avgOve.columns.droplevel(0)
avgOve.reset_index(inplace=True)
# avgOve = avgPropExp.drop(columns=['experiment'])

# avgOve = avgPropExp.groupby(['identifier']).agg({'count':['mean','std']}).fillna(0)
# avgOve.columns = avgOve.columns.droplevel(0)
# avgOve.reset_index(inplace=True)

avgOve.head(10)


,experiment,sum
0,385,338415
1,386,959077
2,389,106497
3,390,2473589
4,391,1681977
5,392,1379082
6,393,1948016
7,394,1843504
8,395,687571
9,396,1428674


In [15]:
finalOve = final.merge(avgOve, on='experiment')
finalOve = finalOve.rename(columns={'sum' : 'messageOverhead'})#.drop(columns=['std'])

finalOve.to_csv('overhead.csv')
finalOve.head(10)

,experiment,start,end,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,bandwidth,messageOverhead
0,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518,unl,8,6,12,4,8,2,0.25,1.0,285.933511,2473589
1,391,2023-10-04 19:25:37.1533399,2023-10-04 19:55:37.1555856,unl,8,6,12,4,8,2,0.25,1.0,195.675311,1681977
2,392,2023-10-04 19:59:12.6589825,2023-10-04 20:29:12.6608307,unl,8,6,12,4,8,2,0.25,1.0,86.268800,1379082
3,393,2023-10-04 20:32:47.3098616,2023-10-04 21:02:47.3149178,unl,8,6,12,4,8,2,0.25,3.0,108.486362,1948016
4,394,2023-10-04 21:06:22.7755634,2023-10-04 21:36:22.7788800,unl,8,6,12,4,8,2,0.25,3.0,94.085382,1843504
5,395,2023-10-04 21:39:58.6528165,2023-10-04 22:09:58.6545536,unl,8,6,12,4,8,2,0.25,3.0,36.928853,687571
6,396,2023-10-04 22:13:33.8146704,2023-10-04 22:43:33.8174094,unl,16,8,20,16,8,4,0.50,1.0,79.777656,1428674
7,397,2023-10-04 22:47:09.3837110,2023-10-04 23:17:09.3868442,unl,16,8,20,16,8,4,0.50,1.0,69.534879,1428254
8,398,2023-10-04 23:20:44.6001719,2023-10-04 23:50:44.6033804,unl,16,8,20,16,8,4,0.50,1.0,7.107765,141417
9,402,2023-10-05 01:35:05.2471450,2023-10-05 02:05:05.2498822,unl,16,8,20,16,8,4,0.50,0.5,75.661552,289177
